# Classification de formes à l'aide d'un réseau de convolution : utilisation des callbacks

> Author: Françoise Bouvet (IJCLab, CNRS)  
> Email: <francoise.bouvet@ijclab.in2p3.fr>

1. [Introduction](#Introduction)
2. [Préparation des données](#Préparation-des-données)
3. [Structure du réseau](#Structure-du-réseau)
4. [Apprentissage](#Apprentissage)
5. [Evaluation](#Evaluation)

## Introduction

L'objectif de ce TP est d'apprendre à utiliser les callbacks, mécanismes très utiles pour suivre l'évolution de l'apprentissage et/ou réguler l'apprentissage. Vous pouvez reprendre votre propre TP précédent et le faire évoluer. 

Les images sont issues de la base de données de la plateforme [kaggle](https://www.kaggle.com/)  

Pour vous aider, vous trouverez des informations complémentaires sur la librairie [keras](https://keras.io/getting_started/)

## Préparation des données

In [ ]:
import numpy as np
from utils import lecture_shape_1channel

rep_data = "../datasets/data_shape/"
lst_shape = ['circle', 'ellipse', 'rectangle', 'square', 'triangle']

# Read input data and transform output into one hot encoding
input_train_raw, output_train_raw = lecture_shape_1channel(rep_data + "train/", "*.png", lst_shape)

if input_train_raw is None or not np.any(input_train_raw):
    print(f'Aucun fichier {extension} trouvé dans {dir}')

In [ ]:
from keras.utils import to_categorical

# Normalize input data
input_train = input_train_raw.astype('float32') / 255.
# Transform output into one hot encoding
output_train = to_categorical(output_train_raw)

# Shuffle input data
ind = np.arange(0, np.shape(input_train)[0])
np.random.shuffle(ind)

# Apply to data ; 
input_train = input_train[ind]
output_train = output_train[ind]

print(f'Il y a {input_train.shape[0]} échantillons')

## Structure du réseau

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout

# Network definition : add successive layers
model = Sequential()

# Convolution layers
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(64, 64, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
#
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#
model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#
model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Reshape input 1D vector
model.add(Flatten())

# Full connected layer (MLP)
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))

# # Output layer
model.add(Dense(5, activation='softmax'))

#### Compilation

In [ ]:
# Compilation of the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

# Display the model
model.summary()

## Apprentissage

#### Callbacks

Un callback est une **tâche effectuée à chaque époque**. Plusieurs callbacks sont **prédéfinis** :

- ReduceLROnPlateau : réduit le learning rate quand une métrique atteint un plateau
- EarlyStopping : arrêt lorsqu’une métrique donnée arrête de progresser
- ModelCheckpoint : sauvegarde régulière du modèle ou des poids
- TensorBoard : pour la visualisation de l’apprentissage avec TensorBoard
- BackupAndRestore : en cas d’interruption
- LearningRateScheduler : appel d’une fonction pour calculer le learning  rate
- RemoteMonitor : envoie de données à un serveur


Il est aussi possible de définir son propre callback. Il dérive de la classe de base **Callback**. 


In [ ]:
from keras.callbacks import Callback

class GenereImage(Callback):

    def __init__(self, model):

        self._model = model

    def on_epoch_end(self, epoch, logs={}):

        n = 2
        if epoch % n == n - 1:
            print(f"C'est l'époque {epoch}  du modèle {model.name}")

In [ ]:
from keras.callbacks import ReduceLROnPlateau, TensorBoard

# Decrease the learning rate factor
reduce_lr_cb = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.0001)
# Suivi de l'appentissage par Tensorboard
tensorboard_cb = TensorBoard(log_dir="./logs")
# Callback personnalisé
custom_cb = GenereImage(model)

cb = [reduce_lr_cb, tensorboard_cb, custom_cb]

history = model.fit(input_train, output_train,
                    epochs=20,
                    validation_split=0.2,
                    batch_size=32,
                    verbose=0,  # pas d'affichage 
                    callbacks=cb)

In [ ]:
from utils import draw_history

draw_history(history)

### Enregistrement du model entraîné pour une utilisation ultérieure

In [ ]:
from keras import models
model.save('./model_shape.keras')

#### Evaluation

In [ ]:
# Read test data
input_test, output_test = lecture_shape_1channel(rep_data + "test/", "*.png", lst_shape)
# Normalize test data
input_test = input_test.astype('float32') / 255.
# Transform output into one hot encoding
output_test = to_categorical(output_test)

# Evaluate the model ; the two parameters are the input_test array and the output_test array
sum_score = model.evaluate(input_test, output_test)
print("Data test : loss %.3f accuracy %.3f" % (sum_score[0], sum_score[1]))

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs